In [1]:
import plotly.express as px
import pandas as pd
from google.cloud import bigquery
import numpy as np
from datetime import time, timedelta, datetime
import math 
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
import scipy.stats as st
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import os
import ipywidgets
import statsmodels.stats.weightstats as sw
from statsmodels.stats.proportion import test_proportions_2indep
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
from google.cloud import bigquery
from scipy.stats import norm
from scipy import stats
pd.options.display.float_format = "{:,.2f}".format
from scipy.stats import norm
from scipy import stats
import statsmodels.stats.weightstats as ws
import openpyxl

# df["Week"] = df["Date"].dt.to_period("W").dt.to_timestamp()

bigquery_client = bigquery.Client(project='analytics-dev-333113')

def cycle_sql(start, end, query, weeks=False):
    """
    You have to use {day} in your script to add cycle date into this backets
    """
    date_start = datetime.strptime(start, '%Y-%m-%d')
    date_end = datetime.strptime(end, '%Y-%m-%d')

    if weeks == False:
        daterange = [(date_start + timedelta(days=x)).strftime('%Y-%m-%d') for x in range(((date_end-date_start).days)+1)]
    else:
        daterange = [(date_start + timedelta(weeks=x)).strftime('%Y-%m-%d') for x in range(((date_end-date_start).days//7)+1)] # weeks dividing days by 7

    total_df = pd.DataFrame()

    for date in daterange:
        print(f"Uploading - {date}:", datetime.today().strftime('%Y-%m-%d %H:%M:%S'))
        script = query.format(date = date)
        df_cycle = bigquery_client.query(script).to_dataframe()
        if df_cycle.empty == True:
            print('Dataframe is empty')
        total_df = pd.concat([df_cycle, total_df])
    return total_df

def read_bq(query, project='analytics-dev-333113'):
    client = bigquery.Client(project=project)
    query_job = client.query(query)
    result_df = query_job.to_dataframe()
    return result_df

def writing_excel(name:str, dataset1=None, dataset2=None, dataset3=None, dataset4=None):
    with pd.ExcelWriter(f"{name}.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index 
    # to store the dataframe in specified sheet

        if dataset1 is not None:
            if dataset2 is not None:
                if dataset3 is not None:
                    if dataset4 is not None:
                        dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                        #   index=False
                                          )
                        dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                        #   index=False
                                          )
                        dataset3.to_excel(writer, sheet_name=f"3-{name}", 
                                        #   index=False
                                          )
                        dataset4.to_excel(writer, sheet_name=f"4-{name}", 
                                        #   index=False
                                          )
                    else:
                        dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                        #   index=False
                                          )
                        dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                        #   index=False
                                          )
                        dataset3.to_excel(writer, sheet_name=f"3-{name}", 
                                        #   index=False
                                          )
                else:
                    dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                    #   index=False
                                      )
                    dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                    #   index=False
                                      )
            else:
                dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                #   index=False
                                  )

        print('DataFrame is written to Excel File successfully.')





# Preparing for future SQL script. Creating conditions for filter to make up the dates when feature was deployed

In [ ]:
df = pd.read_excel('/Users/renatyunison/Downloads/Copy of Actions for violations of standards (1).xlsx', parse_dates = ['Date of launch for push message',
       'Date of launch for bans for late come',
       'Date of launch for bans for other standards',
       'Date of launch for photocheck', 'Comment'])

df = df.replace(['-', 'tbd'], 0)

df_ids = pd.read_excel('/Users/renatyunison/Downloads/Copy of Actions for violations of standards.xlsx')

df_ids = df_ids.dropna()
df_ids['country_id'] = df_ids['country_id'].astype(int)

df['Date of launch for push message'] = pd.to_datetime(df['Date of launch for push message'], format='mixed')
df['Date of launch for bans for late come'] = pd.to_datetime(df['Date of launch for bans for late come'], format='mixed')
df['Date of launch for bans for other standards'] = pd.to_datetime(df['Date of launch for bans for other standards'], format='mixed')
df['Date of launch for photocheck'] = pd.to_datetime(df['Date of launch for photocheck'], format='mixed')

totaldf = pd.merge(df, df_ids, left_on='Country', right_on='country_name')

# list of reviewed countries
y = []
for i in totaldf['country_id']:
    y.append(i)

print(y)

In [ ]:
# creating conditions when for sql with deploying dates

list_pushes = []
list_bans_late = []
list_bans_other_std = []
list_phc = []

for id, dates in zip(totaldf['country_id'], totaldf['Date of launch for push message']):
    otv = f"when country_id = {id} then '{dates}'"
    list_pushes.append(otv)

for id, dates in zip(totaldf['country_id'], totaldf['Date of launch for bans for late come']):
    otv = f"when country_id = {id} then '{dates}'"
    list_bans_late.append(otv)

for id, dates in zip(totaldf['country_id'], totaldf['Date of launch for bans for other standards']):
    otv = f"when country_id = {id} then '{dates}'"
    list_bans_other_std.append(otv)

for id, dates in zip(totaldf['country_id'], totaldf['Date of launch for photocheck']):
    otv = f"when country_id = {id} then '{dates}'"
    list_phc.append(otv)

print('case ')
for sql in list_pushes:
    x = sql.replace('"', "")
    print(sql.replace(' 00:00:00', ""))
print("""ELSE NULL
END
  push_messages,""")

print('case ')
for sql in list_bans_late:
    x = sql.replace('"', "")
    print(sql.replace(' 00:00:00', ""))
print("""ELSE NULL
END
  bans_for_late_come,""")

print('case ')
for sql in list_bans_other_std:
    x = sql.replace('"', "")
    print(sql.replace(' 00:00:00', ""))
print("""ELSE NULL
END
bans_for_other_standards,""")

print('case ')
for sql in list_phc:
    x = sql.replace('"', "")
    print(sql.replace(' 00:00:00', ""))
print("""ELSE NULL
END
  photocontrol,""")


# (Deprecated) (Using BI table which) Drawing chart for all countries with features dates cut-off

In [3]:
query_incident_rate = """
WITH
  cte AS (
  SELECT
    t2.country_id,
    t3.name,
    t.category,
    DATE_TRUNC(t.order_date, week) AS week_order_date,
    SUM(t.incid_count) AS incid_sum,
    CAST(SUM(t.total_rides) AS int) AS total_rides
  FROM
    `indriver-bi.safety.vw_safety_incidents_order_metrics_dash` t
  JOIN
    `indriver-e6e40.ods_monolith.tbl_city` t2
  ON
    t.city_id = t2.id
  JOIN
    `indriver-e6e40.ods_monolith.tbl_country` t3
  ON
    t2.country_id = t3.id
  WHERE
    1=1
    AND confirmation = 'Confirmed'
    AND t2.country_id IN (5,
      40,
      8,
      9,
      53,
      70,
      50,
      54,
      58,
      10,
      165,
      14,
      15,
      86,
      30,
      60,
      84,
      19,
      158,
      75,
      13,
      153,
      46,
      31,
      76,
      11,
      85,
      21,
      23,
      83,
      104,
      22,
      72,
      77,
      43,
      2,
      25,
      71,
      12,
      24,
      6,
      202,
      16,
      69,
      150,
      155,
      78)
  AND order_date between '2024-07-01' and '2024-10-01'
  GROUP BY
    1,
    2,
    3,
    4)
SELECT
  *,
  CASE
    WHEN country_id = 5 THEN '2024-06-17'
    WHEN country_id = 40 THEN '2024-07-02'
    WHEN country_id = 8 THEN '2024-07-02'
    WHEN country_id = 9 THEN '2024-07-02'
    WHEN country_id = 53 THEN '2024-07-02'
    WHEN country_id = 70 THEN '2024-07-02'
    WHEN country_id = 50 THEN '2024-07-02'
    WHEN country_id = 54 THEN '2024-07-02'
    WHEN country_id = 58 THEN '2024-07-02'
    WHEN country_id = 10 THEN '2024-07-02'
    WHEN country_id = 165 THEN '2024-07-22'
    WHEN country_id = 14 THEN '2024-07-22'
    WHEN country_id = 15 THEN '2024-07-22'
    WHEN country_id = 86 THEN '2024-07-22'
    WHEN country_id = 30 THEN '2024-07-22'
    WHEN country_id = 60 THEN '2024-07-29'
    WHEN country_id = 84 THEN '2024-07-29'
    WHEN country_id = 19 THEN '2024-07-29'
    WHEN country_id = 158 THEN '2024-07-29'
    WHEN country_id = 75 THEN '2024-07-29'
    WHEN country_id = 13 THEN '2024-07-29'
    WHEN country_id = 153 THEN '2024-07-29'
    WHEN country_id = 46 THEN '2024-07-29'
    WHEN country_id = 31 THEN '2024-07-29'
    WHEN country_id = 76 THEN '2024-07-29'
    WHEN country_id = 11 THEN '2024-07-29'
    WHEN country_id = 85 THEN '2024-08-05'
    WHEN country_id = 21 THEN '2024-08-05'
    WHEN country_id = 23 THEN '2024-08-05'
    WHEN country_id = 83 THEN '2024-08-05'
    WHEN country_id = 104 THEN '2024-08-05'
    WHEN country_id = 22 THEN '2024-08-05'
    WHEN country_id = 72 THEN '2024-08-05'
    WHEN country_id = 77 THEN '2024-08-05'
    WHEN country_id = 43 THEN '2024-08-05'
    WHEN country_id = 2 THEN '2024-08-05'
    WHEN country_id = 25 THEN '2024-08-05'
    WHEN country_id = 71 THEN '2024-08-05'
    WHEN country_id = 12 THEN '2024-08-05'
    WHEN country_id = 24 THEN '2024-08-05'
    WHEN country_id = 6 THEN '2024-08-05'
    WHEN country_id = 202 THEN '2024-08-05'
    WHEN country_id = 16 THEN '2024-08-05'
    WHEN country_id = 69 THEN '2024-08-05'
    WHEN country_id = 150 THEN '2024-08-05'
    WHEN country_id = 155 THEN '2024-08-05'
    WHEN country_id = 78 THEN '2024-08-05'
    ELSE NULL
END
  push_messages,
  CASE
    WHEN country_id = 5 THEN '2024-06-17'
    WHEN country_id = 40 THEN '2024-07-02'
    WHEN country_id = 8 THEN '2024-07-02'
    WHEN country_id = 9 THEN '2024-07-02'
    WHEN country_id = 53 THEN '2024-07-02'
    WHEN country_id = 70 THEN '2024-07-02'
    WHEN country_id = 50 THEN '2024-07-02'
    WHEN country_id = 54 THEN '2024-07-02'
    WHEN country_id = 58 THEN '2024-07-02'
    WHEN country_id = 10 THEN '2024-07-02'
    WHEN country_id = 165 THEN '2024-07-29'
    WHEN country_id = 14 THEN '2024-07-29'
    WHEN country_id = 15 THEN '2024-07-29'
    WHEN country_id = 86 THEN '2024-07-29'
    WHEN country_id = 30 THEN '2024-07-29'
    WHEN country_id = 60 THEN '2024-08-05'
    WHEN country_id = 84 THEN '2024-08-05'
    WHEN country_id = 19 THEN '2024-08-05'
    WHEN country_id = 158 THEN '2024-08-05'
    WHEN country_id = 75 THEN '2024-08-05'
    WHEN country_id = 13 THEN '2024-08-05'
    WHEN country_id = 153 THEN '2024-08-05'
    WHEN country_id = 46 THEN '2024-08-05'
    WHEN country_id = 31 THEN '2024-08-05'
    WHEN country_id = 76 THEN '2024-08-05'
    WHEN country_id = 11 THEN '2024-08-05'
    WHEN country_id = 85 THEN '2024-08-20'
    WHEN country_id = 21 THEN '2024-08-20'
    WHEN country_id = 23 THEN '2024-08-20'
    WHEN country_id = 83 THEN '2024-08-20'
    WHEN country_id = 104 THEN '2024-08-20'
    WHEN country_id = 22 THEN '2024-08-20'
    WHEN country_id = 72 THEN '2024-08-20'
    WHEN country_id = 77 THEN '2024-08-20'
    WHEN country_id = 43 THEN '2024-08-20'
    WHEN country_id = 2 THEN '2024-08-20'
    WHEN country_id = 25 THEN '2024-08-20'
    WHEN country_id = 71 THEN '2024-08-20'
    WHEN country_id = 12 THEN '2024-08-20'
    WHEN country_id = 24 THEN '2024-08-20'
    WHEN country_id = 6 THEN '2024-08-20'
    WHEN country_id = 202 THEN '2024-08-20'
    WHEN country_id = 16 THEN '2024-08-20'
    WHEN country_id = 69 THEN '2024-08-20'
    WHEN country_id = 150 THEN '2024-08-20'
    WHEN country_id = 155 THEN '2024-08-20'
    WHEN country_id = 78 THEN '2024-08-20'
    ELSE NULL
END
  bans_for_late_come,
  CASE
    WHEN country_id = 5 THEN '2024-06-17'
    WHEN country_id = 40 THEN '2024-07-02'
    WHEN country_id = 8 THEN '2024-07-02'
    WHEN country_id = 9 THEN '2024-07-02'
    WHEN country_id = 53 THEN '2024-07-02'
    WHEN country_id = 70 THEN '2024-07-02'
    WHEN country_id = 50 THEN '2024-07-02'
    WHEN country_id = 54 THEN '2024-07-02'
    WHEN country_id = 58 THEN '2024-07-02'
    WHEN country_id = 10 THEN '2024-07-02'
    WHEN country_id = 165 THEN '2024-07-22'
    WHEN country_id = 14 THEN '2024-07-22'
    WHEN country_id = 15 THEN '2024-07-22'
    WHEN country_id = 86 THEN '2024-07-22'
    WHEN country_id = 30 THEN '2024-07-22'
    WHEN country_id = 60 THEN '2024-07-29'
    WHEN country_id = 84 THEN '2024-07-29'
    WHEN country_id = 19 THEN '2024-07-29'
    WHEN country_id = 158 THEN '2024-07-29'
    WHEN country_id = 75 THEN '2024-07-29'
    WHEN country_id = 13 THEN '2024-07-29'
    WHEN country_id = 153 THEN '2024-07-29'
    WHEN country_id = 46 THEN '2024-07-29'
    WHEN country_id = 31 THEN '2024-07-29'
    WHEN country_id = 76 THEN '2024-07-29'
    WHEN country_id = 11 THEN '2024-07-29'
    WHEN country_id = 85 THEN '2024-08-05'
    WHEN country_id = 21 THEN '2024-08-05'
    WHEN country_id = 23 THEN '2024-08-05'
    WHEN country_id = 83 THEN '2024-08-05'
    WHEN country_id = 104 THEN '2024-08-05'
    WHEN country_id = 22 THEN '2024-08-05'
    WHEN country_id = 72 THEN '2024-08-05'
    WHEN country_id = 77 THEN '2024-08-05'
    WHEN country_id = 43 THEN '2024-08-05'
    WHEN country_id = 2 THEN '2024-08-05'
    WHEN country_id = 25 THEN '2024-08-05'
    WHEN country_id = 71 THEN '2024-08-05'
    WHEN country_id = 12 THEN '2024-08-05'
    WHEN country_id = 24 THEN '2024-08-05'
    WHEN country_id = 6 THEN '2024-08-05'
    WHEN country_id = 202 THEN '2024-08-05'
    WHEN country_id = 16 THEN '2024-08-05'
    WHEN country_id = 69 THEN '2024-08-05'
    WHEN country_id = 150 THEN '2024-08-05'
    WHEN country_id = 155 THEN '2024-08-05'
    WHEN country_id = 78 THEN '2024-08-05'
    ELSE NULL
END
  bans_for_other_standards,
  CASE
    WHEN country_id = 5 THEN '2024-06-17'
    WHEN country_id = 40 THEN '2024-07-02'
    WHEN country_id = 8 THEN '2024-07-02'
    WHEN country_id = 9 THEN '2024-07-02'
    WHEN country_id = 53 THEN '2024-07-02'
    WHEN country_id = 70 THEN '2024-07-02'
    WHEN country_id = 50 THEN '2024-07-02'
    WHEN country_id = 54 THEN '2024-07-02'
    WHEN country_id = 58 THEN '2024-07-02'
    WHEN country_id = 10 THEN '2024-07-02'
    WHEN country_id = 165 THEN '2024-09-05'
    WHEN country_id = 14 THEN '2024-08-20'
    WHEN country_id = 15 THEN '2024-08-20'
    WHEN country_id = 86 THEN '2024-08-20'
    WHEN country_id = 30 THEN '2024-09-05'
    WHEN country_id = 60 THEN '2024-08-20'
    WHEN country_id = 84 THEN '2024-09-05'
    WHEN country_id = 19 THEN '2024-08-20'
    WHEN country_id = 158 THEN '2024-09-05'
    WHEN country_id = 75 THEN '2024-08-20'
    WHEN country_id = 13 THEN '2024-08-20'
    WHEN country_id = 153 THEN '2024-09-05'
    WHEN country_id = 46 THEN '2024-08-20'
    WHEN country_id = 31 THEN '2024-09-05'
    WHEN country_id = 76 THEN '2024-08-20'
    WHEN country_id = 11 THEN '2024-08-20'
    WHEN country_id = 85 THEN '2024-08-20'
    WHEN country_id = 21 THEN '2024-08-20'
    WHEN country_id = 23 THEN '2024-08-20'
    WHEN country_id = 83 THEN '2024-08-20'
    WHEN country_id = 104 THEN '2024-08-20'
    WHEN country_id = 22 THEN '2024-08-20'
    WHEN country_id = 72 THEN '2024-08-20'
    WHEN country_id = 77 THEN '2024-08-20'
    WHEN country_id = 43 THEN '2024-08-20'
    WHEN country_id = 2 THEN '2024-07-18'
    WHEN country_id = 25 THEN '2024-08-20'
    WHEN country_id = 71 THEN '2024-08-20'
    WHEN country_id = 12 THEN '2024-08-20'
    WHEN country_id = 24 THEN '2024-08-20'
    WHEN country_id = 6 THEN '2024-08-20'
    WHEN country_id = 202 THEN '2024-08-20'
    WHEN country_id = 16 THEN '2024-08-20'
    WHEN country_id = 69 THEN '2024-08-20'
    WHEN country_id = 150 THEN '2024-08-20'
    WHEN country_id = 155 THEN '2024-09-05'
    WHEN country_id = 78 THEN '2024-08-20'
    ELSE NULL
END
  photocontrol
FROM
  cte
"""

df_incident_rate = bigquery_client.query(query_incident_rate).to_dataframe()

data_by_dates = df_incident_rate.groupby(['name', 'week_order_date', 'push_messages', 'bans_for_late_come', 'bans_for_other_standards', 'photocontrol'], as_index=False)[['incid_sum', 'total_rides']].agg('sum')
data_by_dates['incid_rate'] = data_by_dates['incid_sum'] / data_by_dates['total_rides'] * 100000

In [ ]:
for city in data_by_dates.name.unique():

  fig = px.line(data_by_dates[['name', 'week_order_date', 'incid_rate']].round(2).query(f"name in ('{city}')")
                , x="week_order_date"
                , y='incid_rate'
                , title=f"Incident rate in {city} / Periods of enabled new features regarding Bans by standards"
                , width=1200, height=600
                , text='incid_rate'
                )

  fig.update_layout(
      xaxis_title="Dates"
                , yaxis_title="Incident rate"
  )

  fig.add_vline(x=datetime.strptime(data_by_dates.query(f"name in ('{city}')")['push_messages'].iloc[0], "%Y-%m-%d").timestamp() * 1000, 
                line_dash="dash",
                annotation_text="Push",
                annotation_position="top left",
                annotation=dict(font_size=10, 
                                font_family="Helvetica"),
                line_color="darkgreen", opacity=0.25, line_width=3
                )
  
  fig.add_vline(x=datetime.strptime(data_by_dates.query(f"name in ('{city}')")['bans_for_late_come'].iloc[0], "%Y-%m-%d").timestamp() * 1000, 
                line_dash="dash",
                annotation_text="Bans for late", 
                annotation_position="bottom left",
                annotation=dict(font_size=10, 
                                font_family="Helvetica"),
                line_color="darkgreen", 
                opacity=0.25, 
                line_width=3)
  
  fig.add_vline(x=datetime.strptime(data_by_dates.query(f"name in ('{city}')")['bans_for_other_standards'].iloc[0], "%Y-%m-%d").timestamp() * 1000, 
                line_dash="dash",
                annotation_text="Bans for other", annotation_position="bottom left",
                annotation=dict(font_size=10, font_family="Helvetica"),
                line_color="darkgreen", 
                line_width=3)
  
  fig.add_vline(x=datetime.strptime(data_by_dates.query(f"name in ('{city}')")['photocontrol'].iloc[0], "%Y-%m-%d").timestamp() * 1000, 
                line_dash="dash",
                annotation_text="Photocontrol", annotation_position="top left",
                annotation=dict(font_size=10, font_family="Helvetica"),
                line_color="darkgreen", 
                line_width=3)

  fig.show()

## Checking the difference & correctness between two tables on Incident Rate

In [85]:
df_incident_rate_first = read_bq("""
WITH jira AS (SELECT DATE_TRUNC(incident_date, WEEK)                               AS incident_date,
                     country_name,
                     COUNT(IF(information_status = 'Confirmed', redmine_id, NULL)) AS cnt_inc
              FROM indriver-bi.safety.vw_safety_incidents_detail
              WHERE 1 = 1
                AND incident_date >= '2024-06-01'
                AND information_status = 'Confirmed'
                AND vertical = 'Incity'
              GROUP BY 1, 2),
     bi AS (SELECT country_name                                           AS country_name_bi,
                   DATE_TRUNC(order_date, WEEK)                           AS order_date,
                   SUM(IF(confirmation = 'Confirmed', incid_count, NULL)) AS incident_count,
            FROM `indriver-bi.safety.vw_safety_incidents_order_metrics_dash`
            WHERE 1 = 1
              AND order_date >= '2024-06-01'
              AND vertical = 'Incity'
              AND confirmation = 'Confirmed'
            GROUP BY 1, 2),
     rides AS (SELECT DATE_TRUNC(order_date_part, WEEK) AS metric_date,
                      t2.name                           AS country_name_rides,
                      t1.country_id,
                      SUM(rides_count_sum)              AS rides
               FROM indriver-bi.incity.tbl_incity_metrics_city_daily t1
                        JOIN indriver-e6e40.ods_monolith.tbl_country t2
                             ON t1.country_id = t2.id
               WHERE order_date_part >= '2024-06-01'
               GROUP BY 1, 2, 3)
SELECT DATE_TRUNC(incident_date, WEEK) AS incident_date_week,
       country_id,
       country_name,
       SUM(incident_count)             AS cnt_inc_bi,
       SUM(cnt_inc)                    AS cnt_inc_initial,
       SUM(rides)                      AS total_rides,
FROM jira t1
         LEFT JOIN bi t2
                   ON t1.incident_date = t2.order_date
                       AND t1.country_name = t2.country_name_bi
         LEFT JOIN rides t3
                   ON t1.incident_date = t3.metric_date
                       AND t1.country_name = t3.country_name_rides
GROUP BY 1, 2, 3 
""")

df_incident_rate_first['incident_rate_bi'] = df_incident_rate_first['cnt_inc_bi'] / df_incident_rate_first['total_rides'] * 100000
df_incident_rate_first['incident_rate_initial'] = df_incident_rate_first['cnt_inc_initial'] / df_incident_rate_first['total_rides'] * 100000

# df_incident_rate_first[df_incident_rate_first['country_name'] == 'Argentina']


In [86]:
for city in df_incident_rate_first['country_name'].unique():

  fig = px.line(df_incident_rate_first.groupby(['incident_date_week', 'country_name'], as_index=False)[['incident_rate_bi', 'incident_rate_initial']].agg('max').round(1).query(f"country_name in ('{city}')")
                , x="incident_date_week"
                , y=['incident_rate_bi', 'incident_rate_initial']
                , title=f"Incident rate in {city}"
                , width=1400, height=600
                )
  
  fig.update_traces(textposition="top center")

  fig.update_layout(
                  xaxis_title="Dates"
                , yaxis_title="Incident rate"
  )


  fig.show()

# (Actual) Drawing chart on Incident Rate by using initial table and new logic as well as cut-offs by new features

### Incident rate with charts and absolute value of metric

In [38]:
df_incident_rate_first = read_bq("""
WITH jira AS (SELECT incident_date,
                     country_name,
                     COUNT(IF(information_status = 'Confirmed', redmine_id, NULL)) AS cnt_inc
              FROM indriver-bi.safety.vw_safety_incidents_detail
              WHERE 1 = 1
                AND incident_date >= '2024-09-01'
                AND information_status = 'Confirmed'
                AND vertical = 'Incity'
              GROUP BY 1, 2),
     bi AS (SELECT country_name                                           AS country_name_bi,
                   order_date,
                   SUM(IF(confirmation = 'Confirmed', incid_count, NULL)) AS incident_count,
            FROM `indriver-bi.safety.vw_safety_incidents_order_metrics_dash`
            WHERE 1 = 1
              AND order_date >= '2024-09-01'
              AND vertical = 'Incity'
              AND confirmation = 'Confirmed'
            GROUP BY 1, 2),
     rides AS (SELECT order_date_part      AS metric_date,
                      t2.name              AS country_name_rides,
                      t1.country_id,
                      SUM(rides_count_sum) AS rides
               FROM indriver-bi.incity.tbl_incity_metrics_city_daily t1
                        JOIN indriver-e6e40.ods_monolith.tbl_country t2
                             ON t1.country_id = t2.id
               WHERE order_date_part >= '2024-09-01'
               GROUP BY 1, 2, 3)
SELECT DATE_TRUNC(incident_date, WEEK) AS incident_date_week,
       country_id,
       country_name,
       SUM(incident_count)             AS cnt_inc_bi,
       SUM(cnt_inc)                    AS cnt_inc_initial,
       SUM(rides)                      AS total_rides,
FROM jira t1
         LEFT JOIN bi t2
                   ON t1.incident_date = t2.order_date
                       AND t1.country_name = t2.country_name_bi
         LEFT JOIN rides t3
                   ON t1.incident_date = t3.metric_date
                       AND t1.country_name = t3.country_name_rides
GROUP BY 1, 2, 3
""")

df_incident_rate_first['incident_rate_bi'] = df_incident_rate_first['cnt_inc_bi'] / df_incident_rate_first['total_rides'] * 100000
df_incident_rate_first['incident_rate_initial'] = df_incident_rate_first['cnt_inc_initial'] / df_incident_rate_first['total_rides'] * 100000

df_incident_rate_first


,incident_date_week,country_id,country_name,cnt_inc_bi,cnt_inc_initial,total_rides,incident_rate_bi,incident_rate_initial
0,2024-09-01,22,Colombia,131,134,3443295,3.80,3.89
1,2024-09-08,<NA>,Non-defined,<NA>,2,<NA>,<NA>,<NA>
2,2024-09-08,12,Mexico,160,163,3701362,4.32,4.40
3,2024-09-15,22,Colombia,127,131,3600715,3.53,3.64
4,2024-10-13,2,Казахстан,77,78,2499779,3.08,3.12
...,...,...,...,...,...,...,...,...
294,2024-10-20,70,Zimbabwe,3,3,20623,14.55,14.55
295,2024-10-13,58,Namibia,1,1,1310,76.34,76.34
296,2024-09-01,6,Украина,1,1,18687,5.35,5.35
297,2024-10-13,8,Kenya,1,1,384,260.42,260.42


In [114]:
for city in df_incident_rate_first.country_name.unique():

  fig = px.line(df_incident_rate_first.groupby(['incident_date_week', 'country_name'], as_index=False)[['incident_rate_bi', 'incident_rate_initial']].agg('max').query(f"country_name in ('{city}')")
                , x="incident_date_week"
                , y=['incident_rate_bi', 'incident_rate_initial']
                , title=f"Incident rate in {city}"
                , width=1500, height=600
                , markers=True
                )

  fig.update_layout(
      xaxis_title="Dates"
                , yaxis_title="Incident rate"
  )

  fig.update_yaxes(rangemode="tozero")

  fig.show()

### Checking total chart on comparison of BI and Initial table

In [7]:
df_total_check = df_incident_rate_first.groupby('incident_date_week', as_index=False)[['cnt_inc_bi', 'cnt_inc_initial', 'total_rides']].agg('sum')

df_total_check['incident_rate_bi'] = df_total_check['cnt_inc_bi'] / df_total_check['total_rides'] * 100000

df_total_check['incident_rate_initial'] = df_total_check['cnt_inc_initial'] / df_total_check['total_rides'] * 100000

In [13]:
df_total_check.groupby('incident_date_week', as_index=False)[['incident_rate_bi', 'incident_rate_initial']].agg('max').rename(columns = {'incident_rate_bi':'BI Tableau table',
                                                                                                                                             'incident_rate_initial':'Original table'})

,incident_date_week,BI Tableau table,Original table
0,2024-04-28,4.40,5.68
1,2024-05-05,4.43,5.86
2,2024-05-12,4.39,5.79
3,2024-05-19,4.76,6.37
4,2024-05-26,4.53,6.30
5,2024-06-02,4.63,6.17
6,2024-06-09,4.50,6.21
7,2024-06-16,4.62,6.28
8,2024-06-23,4.04,5.78
9,2024-06-30,3.81,5.30


In [22]:
fig = px.line(df_total_check.groupby('incident_date_week', as_index=False)[['incident_rate_bi', 'incident_rate_initial']].agg('max').rename(columns = {'incident_rate_bi':'BI Tableau table',
                                                                                                                                             'incident_rate_initial':'Original table'})
        , x="incident_date_week"
        , y=['BI Tableau table', 'Original table']
        , title=f"Incident rate difference between BI Tableau table and initial one"
        , width=1500, height=600
        , markers=True
        )

fig.update_traces(
textposition = 'top right'
)

fig.update_yaxes(rangemode="tozero")


fig.update_layout(
xaxis_title="Periods"
, yaxis_title="Incident Rate"
)

# fig.add_annotation(x='2024-06-16', y=5,
#             text="Text annotation with arrow",
#             showarrow=True,
#             arrowhead=1)

# fig.add_annotation(x='2024-06-16', y=4,
#             text="Text annotation without arrow",
#             showarrow=False,
#             yshift=-20)

In [23]:
test = df_total_check.groupby('incident_date_week', as_index=False)[['incident_rate_bi', 'incident_rate_initial']].agg('max').rename(columns = {'incident_rate_bi':'BI Tableau table',
                                                                                                                                             'incident_rate_initial':'Original table'})

test

,incident_date_week,BI Tableau table,Original table
0,2024-04-28,4.40,5.68
1,2024-05-05,4.43,5.86
2,2024-05-12,4.39,5.79
3,2024-05-19,4.76,6.37
4,2024-05-26,4.53,6.30
5,2024-06-02,4.63,6.17
6,2024-06-09,4.50,6.21
7,2024-06-16,4.62,6.28
8,2024-06-23,4.04,5.78
9,2024-06-30,3.81,5.30


In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Создайте график с plotly.express
fig = px.scatter(test, x='incident_date_week', y=['Original table', 'BI Tableau table'])

# Добавьте текст к выбранным точкам
for i in range(len(test)):
  if test['condition'][i]: # Условие для выбора точек
    fig.add_trace(go.Scatter(
      x=[df['x_column'][i]],
      y=[df['y_column'][i]],
      text=[df['text_column'][i]],
      mode='text+markers',
      marker=dict(size=10, color='red')
    ))

# Удалите исходный след, чтобы оставить только метки
fig.data = fig.data[1:] # Удаление первого следа

fig.show()


### Incident rate diiference Month over month

In [2]:
df_incident_rate_diff = read_bq("""
WITH jira AS (SELECT incident_date,
                     country_name,
                     COUNT(IF(information_status = 'Confirmed', redmine_id, NULL)) AS cnt_inc
              FROM indriver-bi.safety.vw_safety_incidents_detail
              WHERE 1 = 1
                AND incident_date >= '2024-06-01'
                AND information_status = 'Confirmed'
                AND vertical = 'Incity'
              GROUP BY 1, 2),
     bi AS (SELECT country_name                                           AS country_name_bi,
                   order_date,
                   SUM(IF(confirmation = 'Confirmed', incid_count, NULL)) AS incident_count,
            FROM `indriver-bi.safety.vw_safety_incidents_order_metrics_dash`
            WHERE 1 = 1
              AND order_date >= '2024-06-01'
              AND vertical = 'Incity'
              AND confirmation = 'Confirmed'
            GROUP BY 1, 2),
     rides AS (SELECT order_date_part      AS metric_date,
                      t2.name              AS country_name_rides,
                      t1.country_id,
                      SUM(rides_count_sum) AS rides
               FROM indriver-bi.incity.tbl_incity_metrics_city_daily t1
                        JOIN indriver-e6e40.ods_monolith.tbl_country t2
                             ON t1.country_id = t2.id
               WHERE order_date_part >= '2024-06-01'
               GROUP BY 1, 2, 3),
     inc_rate AS
         (SELECT DATE_TRUNC(incident_date, week)             AS incident_date_week,
                 country_id,
                 country_name,
                 SUM(incident_count)                         AS cnt_inc_bi,
                 SUM(cnt_inc)                                AS cnt_inc_initial,
                 SUM(rides)                                  AS total_rides,
                 (SUM(incident_count) / SUM(rides) * 100000) AS inc_rate_bi,
                 (SUM(cnt_inc) / SUM(rides) * 100000)        AS inc_rate_initial
          FROM jira t1
                   LEFT JOIN bi t2
                             ON t1.incident_date = t2.order_date
                                 AND t1.country_name = t2.country_name_bi
                   LEFT JOIN rides t3
                             ON t1.incident_date = t3.metric_date
                                 AND t1.country_name = t3.country_name_rides
          GROUP BY 1, 2, 3)
SELECT *,
       LAG(inc_rate_bi) OVER (PARTITION BY country_name ORDER BY incident_date_week)            AS lag_rate_bi,
       LAG(inc_rate_initial) OVER (PARTITION BY country_name ORDER BY incident_date_week)       AS lag_rate_initial,
       (inc_rate_bi - LAG(inc_rate_bi) OVER (PARTITION BY country_name ORDER BY incident_date_week)) /
       LAG(inc_rate_bi) OVER (PARTITION BY country_name ORDER BY incident_date_week) * 100      AS diff_rate_bi,
       (inc_rate_initial - LAG(inc_rate_initial) OVER (PARTITION BY country_name ORDER BY incident_date_week)) /
       LAG(inc_rate_initial) OVER (PARTITION BY country_name ORDER BY incident_date_week) * 100 AS diff_rate_initial
FROM inc_rate
""")

df_incident_rate_diff['incident_date_week'] = pd.to_datetime(df_incident_rate_diff['incident_date_week'])

In [4]:
df = df_incident_rate_diff.groupby('incident_date_week', as_index=False)[['cnt_inc_initial', 'total_rides']].agg('sum')

df['inc_rate'] = df['cnt_inc_initial'] / df['total_rides'] * 100000

df


,incident_date_week,cnt_inc_initial,total_rides,inc_rate
0,2024-05-26,263,4444879,5.92
1,2024-06-02,1714,27767000,6.17
2,2024-06-09,1754,28246071,6.21
3,2024-06-16,1622,25843652,6.28
4,2024-06-23,1598,27626944,5.78
5,2024-06-30,1502,28352407,5.30
6,2024-07-07,1562,28015640,5.58
7,2024-07-14,1518,27179204,5.59
8,2024-07-21,1443,27150108,5.31
9,2024-07-28,1502,28651109,5.24


In [5]:
df_incident_rate_total = df_incident_rate_diff.query("incident_date_week in ('2024-11-03', '2024-11-10', '2024-11-17', '2024-11-24', ) and country_name not in ('Non-defined')").groupby(['incident_date_week'], as_index=False)[['cnt_inc_initial', 'total_rides']].agg('sum')
                        # .pivot(index='country_name', columns='incident_date_month', values=['inc_rate_initial', 'diff_rate_initial']) \
                        # .fillna('-')

df_incident_rate_total['incident_rate'] = df_incident_rate_total['cnt_inc_initial'] / df_incident_rate_total['total_rides'] * 100000  

df_incident_rate_total

,incident_date_week,cnt_inc_initial,total_rides,incident_rate
0,2024-11-03,1708,31134082,5.49
1,2024-11-10,1592,30577585,5.21
2,2024-11-17,1478,30703750,4.81
3,2024-11-24,847,19790860,4.28


In [6]:
print((5.49 + 5.21)/2, (4.81 + 4.28)/2)

mean1 = (5.49 + 5.21)/2
mean2 = (4.81 + 4.28)/2

print((mean2 - mean1) / mean1 * 100)


5.35 4.545
-15.046728971962612


# Defect rate calculation with charts

### Cycle loading because of size of dataset (tables and query complexity)

In [24]:
query_defect_rate_first = """
WITH quality_logic AS (SELECT DATE_TRUNC(agg_period_dt_part, WEEK) AS date_vio_week,
                              country_id,
                              t2.name                              AS country_name_vio,
                              SUM(standards_violation)             AS violations_vio,
                              SUM(successful_orders)               AS rides_done_vio,
                       FROM (SELECT driver_id,
                                    agg_period_dt_part,
                                    (s11 + s12 + s21 + s22 + s30 + s40 + s51 + s52 + s60 + s70 + s80 + s990) AS standards_violation,
                                    country_id,
                                    successful_orders
                             FROM dwh-storage-327422.emart_service_quality.incity_order_review_drivers_daily
                             WHERE 1 = 1
                               AND country_id IN (SELECT id
                                                  FROM indriver-e6e40.ods_monolith.tbl_country
                                                  WHERE id IN
                                                        (5, 40, 8, 9, 53, 70, 50, 54, 58, 10, 165, 14, 15, 86, 30, 60,
                                                         84,
                                                         19, 158, 75, 13,
                                                         153,
                                                         46, 31, 76, 11, 85, 21, 23, 83, 104, 22, 72, 77, 43, 2, 25, 71,
                                                         12, 24, 6, 202, 16,
                                                         69,
                                                         150, 155, 78))) t1
                                JOIN indriver-e6e40.ods_monolith.tbl_country t2
                                     ON t1.country_id = t2.id
                       WHERE 1 = 1
                         AND DATE_TRUNC(agg_period_dt_part, WEEK) = '{date}'
                       GROUP BY 1, 2, 3),
     rat_logic AS (SELECT DISTINCT DATE_TRUNC(reviews.created_date_part, WEEK) AS date_rating_week,
                                   geo.country_name                            AS country_name_rating,
                                   reviews.visibility,
                                   -- who was reviewed
                                   ROUND(COUNT(DISTINCT
                                               CASE
                                                   WHEN reviews.rating IN (1, 2, 3, 1.0, 2.0, 3.0) THEN reviews.id
                                                   END
                                         ) / COUNT(DISTINCT reviews.id) * 100, 5) defect_rate_rating
                   FROM `indriver-e6e40.ods_review.tbl_review` reviews
                            JOIN
                        `indriver-e6e40.ods_monolith.tbl_user` users
                        ON
                            reviews.target_id = users.id
                            JOIN
                        `indriver-e6e40.heap.vw_macroregion_mapping` geo
                        ON
                            geo.country_id = users.country_id
                                AND DATE_TRUNC(reviews.created_date_part, WEEK) = '{date}'
                   WHERE 1 = 1
                     AND reviews.visibility = 'driver'
                     AND geo.country_name IN ((SELECT name
                                               FROM indriver-e6e40.ods_monolith.tbl_country
                                               WHERE id IN
                                                     (5, 40, 8, 9, 53, 70, 50, 54, 58, 10, 165, 14, 15, 86, 30, 60, 84,
                                                      19, 158, 75, 13,
                                                      153,
                                                      46, 31, 76, 11, 85, 21, 23, 83, 104, 22, 72, 77, 43, 2, 25, 71,
                                                      12, 24, 6, 202, 16,
                                                      69,
                                                      150, 155, 78)))
                   GROUP BY 1,
                            2, 3)
SELECT date_vio_week,
       date_rating_week,
       country_id,
       country_name_vio,
       country_name_rating,
       violations_vio,
       rides_done_vio,
       defect_rate_rating,
       CASE
           WHEN country_id = 5 THEN '2024-06-17'
           WHEN country_id = 40 THEN '2024-07-02'
           WHEN country_id = 8 THEN '2024-07-02'
           WHEN country_id = 9 THEN '2024-07-02'
           WHEN country_id = 53 THEN '2024-07-02'
           WHEN country_id = 70 THEN '2024-07-02'
           WHEN country_id = 50 THEN '2024-07-02'
           WHEN country_id = 54 THEN '2024-07-02'
           WHEN country_id = 58 THEN '2024-07-02'
           WHEN country_id = 10 THEN '2024-07-02'
           WHEN country_id = 165 THEN '2024-07-22'
           WHEN country_id = 14 THEN '2024-07-22'
           WHEN country_id = 15 THEN '2024-07-22'
           WHEN country_id = 86 THEN '2024-07-22'
           WHEN country_id = 30 THEN '2024-07-22'
           WHEN country_id = 60 THEN '2024-07-29'
           WHEN country_id = 84 THEN '2024-07-29'
           WHEN country_id = 19 THEN '2024-07-29'
           WHEN country_id = 158 THEN '2024-07-29'
           WHEN country_id = 75 THEN '2024-07-29'
           WHEN country_id = 13 THEN '2024-07-29'
           WHEN country_id = 153 THEN '2024-07-29'
           WHEN country_id = 46 THEN '2024-07-29'
           WHEN country_id = 31 THEN '2024-07-29'
           WHEN country_id = 76 THEN '2024-07-29'
           WHEN country_id = 11 THEN '2024-07-29'
           WHEN country_id = 85 THEN '2024-08-05'
           WHEN country_id = 21 THEN '2024-08-05'
           WHEN country_id = 23 THEN '2024-08-05'
           WHEN country_id = 83 THEN '2024-08-05'
           WHEN country_id = 104 THEN '2024-08-05'
           WHEN country_id = 22 THEN '2024-08-05'
           WHEN country_id = 72 THEN '2024-08-05'
           WHEN country_id = 77 THEN '2024-08-05'
           WHEN country_id = 43 THEN '2024-08-05'
           WHEN country_id = 2 THEN '2024-08-05'
           WHEN country_id = 25 THEN '2024-08-05'
           WHEN country_id = 71 THEN '2024-08-05'
           WHEN country_id = 12 THEN '2024-08-05'
           WHEN country_id = 24 THEN '2024-08-05'
           WHEN country_id = 6 THEN '2024-08-05'
           WHEN country_id = 202 THEN '2024-08-05'
           WHEN country_id = 16 THEN '2024-08-05'
           WHEN country_id = 69 THEN '2024-08-05'
           WHEN country_id = 150 THEN '2024-08-05'
           WHEN country_id = 155 THEN '2024-08-05'
           WHEN country_id = 78 THEN '2024-08-05'
           ELSE NULL
           END
           push_messages,
       CASE
           WHEN country_id = 5 THEN '2024-06-17'
           WHEN country_id = 40 THEN '2024-07-02'
           WHEN country_id = 8 THEN '2024-07-02'
           WHEN country_id = 9 THEN '2024-07-02'
           WHEN country_id = 53 THEN '2024-07-02'
           WHEN country_id = 70 THEN '2024-07-02'
           WHEN country_id = 50 THEN '2024-07-02'
           WHEN country_id = 54 THEN '2024-07-02'
           WHEN country_id = 58 THEN '2024-07-02'
           WHEN country_id = 10 THEN '2024-07-02'
           WHEN country_id = 165 THEN '2024-07-29'
           WHEN country_id = 14 THEN '2024-07-29'
           WHEN country_id = 15 THEN '2024-07-29'
           WHEN country_id = 86 THEN '2024-07-29'
           WHEN country_id = 30 THEN '2024-07-29'
           WHEN country_id = 60 THEN '2024-08-05'
           WHEN country_id = 84 THEN '2024-08-05'
           WHEN country_id = 19 THEN '2024-08-05'
           WHEN country_id = 158 THEN '2024-08-05'
           WHEN country_id = 75 THEN '2024-08-05'
           WHEN country_id = 13 THEN '2024-08-05'
           WHEN country_id = 153 THEN '2024-08-05'
           WHEN country_id = 46 THEN '2024-08-05'
           WHEN country_id = 31 THEN '2024-08-05'
           WHEN country_id = 76 THEN '2024-08-05'
           WHEN country_id = 11 THEN '2024-08-05'
           WHEN country_id = 85 THEN '2024-08-20'
           WHEN country_id = 21 THEN '2024-08-20'
           WHEN country_id = 23 THEN '2024-08-20'
           WHEN country_id = 83 THEN '2024-08-20'
           WHEN country_id = 104 THEN '2024-08-20'
           WHEN country_id = 22 THEN '2024-08-20'
           WHEN country_id = 72 THEN '2024-08-20'
           WHEN country_id = 77 THEN '2024-08-20'
           WHEN country_id = 43 THEN '2024-08-20'
           WHEN country_id = 2 THEN '2024-08-20'
           WHEN country_id = 25 THEN '2024-08-20'
           WHEN country_id = 71 THEN '2024-08-20'
           WHEN country_id = 12 THEN '2024-08-20'
           WHEN country_id = 24 THEN '2024-08-20'
           WHEN country_id = 6 THEN '2024-08-20'
           WHEN country_id = 202 THEN '2024-08-20'
           WHEN country_id = 16 THEN '2024-08-20'
           WHEN country_id = 69 THEN '2024-08-20'
           WHEN country_id = 150 THEN '2024-08-20'
           WHEN country_id = 155 THEN '2024-08-20'
           WHEN country_id = 78 THEN '2024-08-20'
           ELSE NULL
           END
           bans_for_late_come,
       CASE
           WHEN country_id = 5 THEN '2024-06-17'
           WHEN country_id = 40 THEN '2024-07-02'
           WHEN country_id = 8 THEN '2024-07-02'
           WHEN country_id = 9 THEN '2024-07-02'
           WHEN country_id = 53 THEN '2024-07-02'
           WHEN country_id = 70 THEN '2024-07-02'
           WHEN country_id = 50 THEN '2024-07-02'
           WHEN country_id = 54 THEN '2024-07-02'
           WHEN country_id = 58 THEN '2024-07-02'
           WHEN country_id = 10 THEN '2024-07-02'
           WHEN country_id = 165 THEN '2024-07-22'
           WHEN country_id = 14 THEN '2024-07-22'
           WHEN country_id = 15 THEN '2024-07-22'
           WHEN country_id = 86 THEN '2024-07-22'
           WHEN country_id = 30 THEN '2024-07-22'
           WHEN country_id = 60 THEN '2024-07-29'
           WHEN country_id = 84 THEN '2024-07-29'
           WHEN country_id = 19 THEN '2024-07-29'
           WHEN country_id = 158 THEN '2024-07-29'
           WHEN country_id = 75 THEN '2024-07-29'
           WHEN country_id = 13 THEN '2024-07-29'
           WHEN country_id = 153 THEN '2024-07-29'
           WHEN country_id = 46 THEN '2024-07-29'
           WHEN country_id = 31 THEN '2024-07-29'
           WHEN country_id = 76 THEN '2024-07-29'
           WHEN country_id = 11 THEN '2024-07-29'
           WHEN country_id = 85 THEN '2024-08-05'
           WHEN country_id = 21 THEN '2024-08-05'
           WHEN country_id = 23 THEN '2024-08-05'
           WHEN country_id = 83 THEN '2024-08-05'
           WHEN country_id = 104 THEN '2024-08-05'
           WHEN country_id = 22 THEN '2024-08-05'
           WHEN country_id = 72 THEN '2024-08-05'
           WHEN country_id = 77 THEN '2024-08-05'
           WHEN country_id = 43 THEN '2024-08-05'
           WHEN country_id = 2 THEN '2024-08-05'
           WHEN country_id = 25 THEN '2024-08-05'
           WHEN country_id = 71 THEN '2024-08-05'
           WHEN country_id = 12 THEN '2024-08-05'
           WHEN country_id = 24 THEN '2024-08-05'
           WHEN country_id = 6 THEN '2024-08-05'
           WHEN country_id = 202 THEN '2024-08-05'
           WHEN country_id = 16 THEN '2024-08-05'
           WHEN country_id = 69 THEN '2024-08-05'
           WHEN country_id = 150 THEN '2024-08-05'
           WHEN country_id = 155 THEN '2024-08-05'
           WHEN country_id = 78 THEN '2024-08-05'
           ELSE NULL
           END
           bans_for_other_standards,
       CASE
           WHEN country_id = 5 THEN '2024-06-17'
           WHEN country_id = 40 THEN '2024-07-02'
           WHEN country_id = 8 THEN '2024-07-02'
           WHEN country_id = 9 THEN '2024-07-02'
           WHEN country_id = 53 THEN '2024-07-02'
           WHEN country_id = 70 THEN '2024-07-02'
           WHEN country_id = 50 THEN '2024-07-02'
           WHEN country_id = 54 THEN '2024-07-02'
           WHEN country_id = 58 THEN '2024-07-02'
           WHEN country_id = 10 THEN '2024-07-02'
           WHEN country_id = 165 THEN '2024-09-05'
           WHEN country_id = 14 THEN '2024-08-20'
           WHEN country_id = 15 THEN '2024-08-20'
           WHEN country_id = 86 THEN '2024-08-20'
           WHEN country_id = 30 THEN '2024-09-05'
           WHEN country_id = 60 THEN '2024-08-20'
           WHEN country_id = 84 THEN '2024-09-05'
           WHEN country_id = 19 THEN '2024-08-20'
           WHEN country_id = 158 THEN '2024-09-05'
           WHEN country_id = 75 THEN '2024-08-20'
           WHEN country_id = 13 THEN '2024-08-20'
           WHEN country_id = 153 THEN '2024-09-05'
           WHEN country_id = 46 THEN '2024-08-20'
           WHEN country_id = 31 THEN '2024-09-05'
           WHEN country_id = 76 THEN '2024-08-20'
           WHEN country_id = 11 THEN '2024-08-20'
           WHEN country_id = 85 THEN '2024-08-20'
           WHEN country_id = 21 THEN '2024-08-20'
           WHEN country_id = 23 THEN '2024-08-20'
           WHEN country_id = 83 THEN '2024-08-20'
           WHEN country_id = 104 THEN '2024-08-20'
           WHEN country_id = 22 THEN '2024-08-20'
           WHEN country_id = 72 THEN '2024-08-20'
           WHEN country_id = 77 THEN '2024-08-20'
           WHEN country_id = 43 THEN '2024-08-20'
           WHEN country_id = 2 THEN '2024-07-18'
           WHEN country_id = 25 THEN '2024-08-20'
           WHEN country_id = 71 THEN '2024-08-20'
           WHEN country_id = 12 THEN '2024-08-20'
           WHEN country_id = 24 THEN '2024-08-20'
           WHEN country_id = 6 THEN '2024-08-20'
           WHEN country_id = 202 THEN '2024-08-20'
           WHEN country_id = 16 THEN '2024-08-20'
           WHEN country_id = 69 THEN '2024-08-20'
           WHEN country_id = 150 THEN '2024-08-20'
           WHEN country_id = 155 THEN '2024-09-05'
           WHEN country_id = 78 THEN '2024-08-20'
           ELSE NULL
           END
           photocontrol
FROM quality_logic t1
         LEFT JOIN rat_logic t2
                   ON t1.date_vio_week = t2.date_rating_week
                       AND t1.country_name_vio = t2.country_name_rating
"""

df = cycle_sql('2024-06-30', '2024-10-13', query_defect_rate_first, weeks=True)
df['defect_rate'] = df['violations_vio'] / df['rides_done_vio']

Uploading - 2024-06-30: 2024-10-17 12:48:48
Uploading - 2024-07-07: 2024-10-17 12:51:58


KeyboardInterrupt: 

In [ ]:
df

In [ ]:
for city in df['country_name_vio'].unique():

  fig = px.line(df.groupby(['date_vio_week', 'country_name_vio'], as_index=False)[['defect_rate', 'defect_rate_rating']].agg('max').query(f"country_name_vio in ('{city}')")
                , x="date_vio_week"
                , y=['defect_rate', 'defect_rate_rating']
                , title=f"Defect rate in {city}"
                , width=1500, height=600
                , markers=True
                )
  
  fig.update_traces(
    textposition = 'top right'
  )

  fig.update_yaxes(rangemode="tozero")


  fig.update_layout(
      xaxis_title="Dates"
      , yaxis_title="Defect rate"
  )


  fig.add_vline(x=datetime.strptime(df.query(f"country_name_vio in ('{city}')")['push_messages'].iloc[0], "%Y-%m-%d").timestamp() * 1000, 
                line_dash="dash",
                annotation_text="Push",
                annotation_position="top right",
                annotation=dict(font_size=13, 
                                font_family="Helvetica",
                                ),
                line_color="gray", 
                line_width=3
                )
  
  fig.add_vline(x=datetime.strptime(df.query(f"country_name_vio in ('{city}')")['bans_for_late_come'].iloc[0], "%Y-%m-%d").timestamp() * 1000, 
                line_dash="dash",
                annotation_text="Bans for late", 
                annotation_position="bottom right",
                annotation=dict(font_size=13, 
                                font_family="Helvetica",
                                ),
                line_color="gray", 
                line_width=3)
  
  fig.add_vline(x=datetime.strptime(df.query(f"country_name_vio in ('{city}')")['bans_for_other_standards'].iloc[0], "%Y-%m-%d").timestamp() * 1000, 
                line_dash="dash",
                annotation_text="Bans for other", 
                annotation_position="bottom left",
                annotation=dict(font_size=13, 
                                font_family="Helvetica",
                                ),
                line_color="gray", 
                line_width=3)
  
  fig.add_vline(x=datetime.strptime(df.query(f"country_name_vio in ('{city}')")['photocontrol'].iloc[0], "%Y-%m-%d").timestamp() * 1000, 
                line_dash="dash",
                annotation_text="Photocontrol", 
                annotation_position="top left",
                annotation=dict(font_size=13, 
                                font_family="Helvetica",
                                ),
                line_color="gray", 
                line_width=3)

  fig.show()

### Loading quality team logic

In [21]:
df_defect_rate_quality = read_bq("""
WITH cte AS (SELECT 
                                date_trunc(agg_period_dt_part, week)       AS date_vio_week,
                                 -- agg_period_dt_part AS date_vio,
                    country_id,
                    t2.name                  AS country_name_vio,
                    SUM(standards_violation) AS violations,
                    SUM(accepted_orders)     AS accepted_orders,
                    CASE
                        WHEN country_id = 5 THEN '2024-06-17'
                        WHEN country_id = 40 THEN '2024-07-02'
                        WHEN country_id = 8 THEN '2024-07-02'
                        WHEN country_id = 9 THEN '2024-07-02'
                        WHEN country_id = 53 THEN '2024-07-02'
                        WHEN country_id = 70 THEN '2024-07-02'
                        WHEN country_id = 50 THEN '2024-07-02'
                        WHEN country_id = 54 THEN '2024-07-02'
                        WHEN country_id = 58 THEN '2024-07-02'
                        WHEN country_id = 10 THEN '2024-07-02'
                        WHEN country_id = 165 THEN '2024-07-22'
                        WHEN country_id = 14 THEN '2024-07-22'
                        WHEN country_id = 15 THEN '2024-07-22'
                        WHEN country_id = 86 THEN '2024-07-22'
                        WHEN country_id = 30 THEN '2024-07-22'
                        WHEN country_id = 60 THEN '2024-07-29'
                        WHEN country_id = 84 THEN '2024-07-29'
                        WHEN country_id = 19 THEN '2024-07-29'
                        WHEN country_id = 158 THEN '2024-07-29'
                        WHEN country_id = 75 THEN '2024-07-29'
                        WHEN country_id = 13 THEN '2024-07-29'
                        WHEN country_id = 153 THEN '2024-07-29'
                        WHEN country_id = 46 THEN '2024-07-29'
                        WHEN country_id = 31 THEN '2024-07-29'
                        WHEN country_id = 76 THEN '2024-07-29'
                        WHEN country_id = 11 THEN '2024-07-29'
                        WHEN country_id = 85 THEN '2024-08-05'
                        WHEN country_id = 21 THEN '2024-08-05'
                        WHEN country_id = 23 THEN '2024-08-05'
                        WHEN country_id = 83 THEN '2024-08-05'
                        WHEN country_id = 104 THEN '2024-08-05'
                        WHEN country_id = 22 THEN '2024-08-05'
                        WHEN country_id = 72 THEN '2024-08-05'
                        WHEN country_id = 77 THEN '2024-08-05'
                        WHEN country_id = 43 THEN '2024-08-05'
                        WHEN country_id = 2 THEN '2024-08-05'
                        WHEN country_id = 25 THEN '2024-08-05'
                        WHEN country_id = 71 THEN '2024-08-05'
                        WHEN country_id = 12 THEN '2024-08-05'
                        WHEN country_id = 24 THEN '2024-08-05'
                        WHEN country_id = 6 THEN '2024-08-05'
                        WHEN country_id = 202 THEN '2024-08-05'
                        WHEN country_id = 16 THEN '2024-08-05'
                        WHEN country_id = 69 THEN '2024-08-05'
                        WHEN country_id = 150 THEN '2024-08-05'
                        WHEN country_id = 155 THEN '2024-08-05'
                        WHEN country_id = 78 THEN '2024-08-05'
                        ELSE NULL
                        END
                                                push_messages,
                    CASE
                        WHEN country_id = 5 THEN '2024-06-17'
                        WHEN country_id = 40 THEN '2024-07-02'
                        WHEN country_id = 8 THEN '2024-07-02'
                        WHEN country_id = 9 THEN '2024-07-02'
                        WHEN country_id = 53 THEN '2024-07-02'
                        WHEN country_id = 70 THEN '2024-07-02'
                        WHEN country_id = 50 THEN '2024-07-02'
                        WHEN country_id = 54 THEN '2024-07-02'
                        WHEN country_id = 58 THEN '2024-07-02'
                        WHEN country_id = 10 THEN '2024-07-02'
                        WHEN country_id = 165 THEN '2024-07-29'
                        WHEN country_id = 14 THEN '2024-07-29'
                        WHEN country_id = 15 THEN '2024-07-29'
                        WHEN country_id = 86 THEN '2024-07-29'
                        WHEN country_id = 30 THEN '2024-07-29'
                        WHEN country_id = 60 THEN '2024-08-05'
                        WHEN country_id = 84 THEN '2024-08-05'
                        WHEN country_id = 19 THEN '2024-08-05'
                        WHEN country_id = 158 THEN '2024-08-05'
                        WHEN country_id = 75 THEN '2024-08-05'
                        WHEN country_id = 13 THEN '2024-08-05'
                        WHEN country_id = 153 THEN '2024-08-05'
                        WHEN country_id = 46 THEN '2024-08-05'
                        WHEN country_id = 31 THEN '2024-08-05'
                        WHEN country_id = 76 THEN '2024-08-05'
                        WHEN country_id = 11 THEN '2024-08-05'
                        WHEN country_id = 85 THEN '2024-08-20'
                        WHEN country_id = 21 THEN '2024-08-20'
                        WHEN country_id = 23 THEN '2024-08-20'
                        WHEN country_id = 83 THEN '2024-08-20'
                        WHEN country_id = 104 THEN '2024-08-20'
                        WHEN country_id = 22 THEN '2024-08-20'
                        WHEN country_id = 72 THEN '2024-08-20'
                        WHEN country_id = 77 THEN '2024-08-20'
                        WHEN country_id = 43 THEN '2024-08-20'
                        WHEN country_id = 2 THEN '2024-08-20'
                        WHEN country_id = 25 THEN '2024-08-20'
                        WHEN country_id = 71 THEN '2024-08-20'
                        WHEN country_id = 12 THEN '2024-08-20'
                        WHEN country_id = 24 THEN '2024-08-20'
                        WHEN country_id = 6 THEN '2024-08-20'
                        WHEN country_id = 202 THEN '2024-08-20'
                        WHEN country_id = 16 THEN '2024-08-20'
                        WHEN country_id = 69 THEN '2024-08-20'
                        WHEN country_id = 150 THEN '2024-08-20'
                        WHEN country_id = 155 THEN '2024-08-20'
                        WHEN country_id = 78 THEN '2024-08-20'
                        ELSE NULL
                        END
                                                bans_for_late_come,
                    CASE
                        WHEN country_id = 5 THEN '2024-06-17'
                        WHEN country_id = 40 THEN '2024-07-02'
                        WHEN country_id = 8 THEN '2024-07-02'
                        WHEN country_id = 9 THEN '2024-07-02'
                        WHEN country_id = 53 THEN '2024-07-02'
                        WHEN country_id = 70 THEN '2024-07-02'
                        WHEN country_id = 50 THEN '2024-07-02'
                        WHEN country_id = 54 THEN '2024-07-02'
                        WHEN country_id = 58 THEN '2024-07-02'
                        WHEN country_id = 10 THEN '2024-07-02'
                        WHEN country_id = 165 THEN '2024-07-22'
                        WHEN country_id = 14 THEN '2024-07-22'
                        WHEN country_id = 15 THEN '2024-07-22'
                        WHEN country_id = 86 THEN '2024-07-22'
                        WHEN country_id = 30 THEN '2024-07-22'
                        WHEN country_id = 60 THEN '2024-07-29'
                        WHEN country_id = 84 THEN '2024-07-29'
                        WHEN country_id = 19 THEN '2024-07-29'
                        WHEN country_id = 158 THEN '2024-07-29'
                        WHEN country_id = 75 THEN '2024-07-29'
                        WHEN country_id = 13 THEN '2024-07-29'
                        WHEN country_id = 153 THEN '2024-07-29'
                        WHEN country_id = 46 THEN '2024-07-29'
                        WHEN country_id = 31 THEN '2024-07-29'
                        WHEN country_id = 76 THEN '2024-07-29'
                        WHEN country_id = 11 THEN '2024-07-29'
                        WHEN country_id = 85 THEN '2024-08-05'
                        WHEN country_id = 21 THEN '2024-08-05'
                        WHEN country_id = 23 THEN '2024-08-05'
                        WHEN country_id = 83 THEN '2024-08-05'
                        WHEN country_id = 104 THEN '2024-08-05'
                        WHEN country_id = 22 THEN '2024-08-05'
                        WHEN country_id = 72 THEN '2024-08-05'
                        WHEN country_id = 77 THEN '2024-08-05'
                        WHEN country_id = 43 THEN '2024-08-05'
                        WHEN country_id = 2 THEN '2024-08-05'
                        WHEN country_id = 25 THEN '2024-08-05'
                        WHEN country_id = 71 THEN '2024-08-05'
                        WHEN country_id = 12 THEN '2024-08-05'
                        WHEN country_id = 24 THEN '2024-08-05'
                        WHEN country_id = 6 THEN '2024-08-05'
                        WHEN country_id = 202 THEN '2024-08-05'
                        WHEN country_id = 16 THEN '2024-08-05'
                        WHEN country_id = 69 THEN '2024-08-05'
                        WHEN country_id = 150 THEN '2024-08-05'
                        WHEN country_id = 155 THEN '2024-08-05'
                        WHEN country_id = 78 THEN '2024-08-05'
                        ELSE NULL
                        END
                                                bans_for_other_standards,
                    CASE
                        WHEN country_id = 5 THEN '2024-06-17'
                        WHEN country_id = 40 THEN '2024-07-02'
                        WHEN country_id = 8 THEN '2024-07-02'
                        WHEN country_id = 9 THEN '2024-07-02'
                        WHEN country_id = 53 THEN '2024-07-02'
                        WHEN country_id = 70 THEN '2024-07-02'
                        WHEN country_id = 50 THEN '2024-07-02'
                        WHEN country_id = 54 THEN '2024-07-02'
                        WHEN country_id = 58 THEN '2024-07-02'
                        WHEN country_id = 10 THEN '2024-07-02'
                        WHEN country_id = 165 THEN '2024-09-05'
                        WHEN country_id = 14 THEN '2024-08-20'
                        WHEN country_id = 15 THEN '2024-08-20'
                        WHEN country_id = 86 THEN '2024-08-20'
                        WHEN country_id = 30 THEN '2024-09-05'
                        WHEN country_id = 60 THEN '2024-08-20'
                        WHEN country_id = 84 THEN '2024-09-05'
                        WHEN country_id = 19 THEN '2024-08-20'
                        WHEN country_id = 158 THEN '2024-09-05'
                        WHEN country_id = 75 THEN '2024-08-20'
                        WHEN country_id = 13 THEN '2024-08-20'
                        WHEN country_id = 153 THEN '2024-09-05'
                        WHEN country_id = 46 THEN '2024-08-20'
                        WHEN country_id = 31 THEN '2024-09-05'
                        WHEN country_id = 76 THEN '2024-08-20'
                        WHEN country_id = 11 THEN '2024-08-20'
                        WHEN country_id = 85 THEN '2024-08-20'
                        WHEN country_id = 21 THEN '2024-08-20'
                        WHEN country_id = 23 THEN '2024-08-20'
                        WHEN country_id = 83 THEN '2024-08-20'
                        WHEN country_id = 104 THEN '2024-08-20'
                        WHEN country_id = 22 THEN '2024-08-20'
                        WHEN country_id = 72 THEN '2024-08-20'
                        WHEN country_id = 77 THEN '2024-08-20'
                        WHEN country_id = 43 THEN '2024-08-20'
                        WHEN country_id = 2 THEN '2024-07-18'
                        WHEN country_id = 25 THEN '2024-08-20'
                        WHEN country_id = 71 THEN '2024-08-20'
                        WHEN country_id = 12 THEN '2024-08-20'
                        WHEN country_id = 24 THEN '2024-08-20'
                        WHEN country_id = 6 THEN '2024-08-20'
                        WHEN country_id = 202 THEN '2024-08-20'
                        WHEN country_id = 16 THEN '2024-08-20'
                        WHEN country_id = 69 THEN '2024-08-20'
                        WHEN country_id = 150 THEN '2024-08-20'
                        WHEN country_id = 155 THEN '2024-09-05'
                        WHEN country_id = 78 THEN '2024-08-20'
                        ELSE NULL
                        END
                                                photocontrol
             FROM (SELECT driver_id,
                          agg_period_dt_part,
                          (s11 + s12 + s21 + s22 + s121 + s30 + s40 + s51 + s52 + s60 + s70 + s80 + s990) AS standards_violation,
                          country_id,
                          accepted_orders
                   FROM dwh-storage-327422.emart_service_quality.incity_order_review_drivers_daily
                   WHERE 1 = 1
                     AND agg_period_dt_part >= '2024-06-30'
                     AND country_id IN (SELECT id
                                        FROM indriver-e6e40.ods_monolith.tbl_country
                                        WHERE id IN
                                              (5, 40, 8, 9, 53, 70, 50, 54, 58, 10, 165, 14, 15, 86, 30, 60,
                                               84,
                                               19, 158, 75, 13,
                                               153,
                                               46, 31, 76, 11, 85, 21, 23, 83, 104, 22, 72, 77, 43, 2, 25, 71,
                                               12, 24, 6, 202, 16,
                                               69,
                                               150, 155, 78))) t1
                      JOIN indriver-e6e40.ods_monolith.tbl_country t2
                           ON t1.country_id = t2.id
             GROUP BY 1, 2, 3)
SELECT *
FROM cte
""")

df_defect_rate_quality['date_vio_week'] = pd.to_datetime(df_defect_rate_quality['date_vio_week'])

df_defect_rate_quality['defect_rate'] = (df_defect_rate_quality['violations'] / df_defect_rate_quality['accepted_orders']).round(3)

df_defect_rate_quality.head(10)

,date_vio_week,country_id,country_name_vio,violations,accepted_orders,push_messages,bans_for_late_come,bans_for_other_standards,photocontrol,defect_rate
0,2024-08-18,31,Tunisia,7599,41520,2024-07-29,2024-08-05,2024-07-29,2024-09-05,0.18
1,2024-08-18,84,Turkey,9764,49160,2024-07-29,2024-08-05,2024-07-29,2024-09-05,0.20
2,2024-09-22,69,Zambia,601,2192,2024-08-05,2024-08-20,2024-08-05,2024-08-20,0.27
3,2024-09-22,78,Paraguay,8,26,2024-08-05,2024-08-20,2024-08-05,2024-08-20,0.31
4,2024-09-15,14,India,556359,1709201,2024-07-22,2024-07-29,2024-07-22,2024-08-20,0.33
5,2024-09-08,54,Jamaica,16371,48913,2024-07-02,2024-07-02,2024-07-02,2024-07-02,0.34
6,2024-10-20,9,Tanzania,1438,5183,2024-07-02,2024-07-02,2024-07-02,2024-07-02,0.28
7,2024-10-20,14,India,517821,1602914,2024-07-22,2024-07-29,2024-07-22,2024-08-20,0.32
8,2024-10-20,53,Ghana,472,1175,2024-07-02,2024-07-02,2024-07-02,2024-07-02,0.40
9,2024-09-01,16,Philippines,18252,51552,2024-08-05,2024-08-20,2024-08-05,2024-08-20,0.35


In [25]:
df = df_defect_rate_quality.groupby('date_vio_week', as_index=False)[['violations', 'accepted_orders']].agg('sum')

df['defect_rate'] = df['violations'] / df['accepted_orders'] * 100

df.query("date_vio_week in ('2024-10-20', '2024-10-27', '2024-11-03', '2024-11-10')").sort_values('date_vio_week', ascending=True)

,date_vio_week,violations,accepted_orders,defect_rate
16,2024-10-20,8587136,38826039,22.12
17,2024-10-27,8906179,40113572,22.20
18,2024-11-03,8916695,40362458,22.09
19,2024-11-10,8786293,40136547,21.89


In [28]:
print((22.12 + 22.20)/2, ' <- 1 period & 2 period -> ', (22.09 + 21.89)/2)

mean1 = (22.12 + 22.20)/2
mean2 = (22.09 + 21.89)/2

print((mean2 - mean1) / mean1 * 100)


22.16  <- 1 period & 2 period ->  21.990000000000002
-0.7671480144404249


In [52]:
for city in df_defect_rate_quality['country_name_vio'].unique():

  fig = px.line(df_defect_rate_quality.groupby(['date_vio', 'country_name_vio', 'push_messages', 'bans_for_late_come',	'bans_for_other_standards',	'photocontrol'], as_index=False)['defect_rate'] \
                        .agg('max').query(f"country_name_vio in ('{city}')")
                , x="date_vio"
                , y='defect_rate'
                , title=f"Defect rate in {city}"
                , text = 'defect_rate'
                , width=1500, height=600
                , markers=True
                )
  
  fig.update_traces(
    textposition = 'top right'
  )

  fig.update_yaxes(rangemode="tozero")


  fig.update_layout(
      xaxis_title="Dates"
      , yaxis_title="Defect rate"
  )


  fig.add_vline(x=datetime.strptime(df_defect_rate_quality.query(f"country_name_vio in ('{city}')")['push_messages'].iloc[0], "%Y-%m-%d").timestamp() * 1000, 
                line_dash="dash",
                annotation_text="Push",
                annotation_position="top left",
                annotation=dict(font_size=13, 
                                font_family="Helvetica"),
                line_color="grey", 
                line_width=3
                )
  
  fig.add_vline(x=datetime.strptime(df_defect_rate_quality.query(f"country_name_vio in ('{city}')")['bans_for_late_come'].iloc[0], "%Y-%m-%d").timestamp() * 1000, 
                line_dash="dash",
                annotation_text="Bans for late", 
                annotation_position="bottom left",
                annotation=dict(font_size=13, 
                                font_family="Helvetica"),
                line_color="grey", 
                line_width=3)
  
  fig.add_vline(x=datetime.strptime(df_defect_rate_quality.query(f"country_name_vio in ('{city}')")['bans_for_other_standards'].iloc[0], "%Y-%m-%d").timestamp() * 1000, 
                line_dash="dash",
                annotation_text="Bans for other", annotation_position="bottom left",
                annotation=dict(font_size=13, 
                                font_family="Helvetica"),
                line_color="grey", 
                line_width=3)
  
  fig.add_vline(x=datetime.strptime(df_defect_rate_quality.query(f"country_name_vio in ('{city}')")['photocontrol'].iloc[0], "%Y-%m-%d").timestamp() * 1000, 
                line_dash="dash",
                annotation_text="Photocontrol", annotation_position="top left",
                annotation=dict(font_size=13, 
                                font_family="Helvetica"),
                line_color="grey", 
                line_width=3)

  fig.show()

### Defect rate for all countries by dates

In [57]:
df_agg_week = df_defect_rate_quality.groupby('date_vio_week', as_index=False)[['violations', 'accepted_orders']].agg('sum')

df_agg_week['Defect rate'] = df_agg_week['violations'] / df_agg_week['accepted_orders'] * 100

In [59]:
df_agg_week['Defect rate']

0    23.11
1    23.11
2    22.84
3    23.11
4    22.95
5    22.75
6    22.78
7    22.73
8    22.45
9    22.77
10   22.43
11   22.17
12   22.76
13   22.45
14   22.09
15   21.70
Name: Defect rate, dtype: Float64

In [84]:
fig = px.line(df_agg
        , x="date_vio"
        , y='Defect rate'
        , title=f"Defect rate / All regions / Q3"
        , width=1500, height=600
        , markers=True
        # , color_discrete_map={"Defect rate": "gray"}
        )

fig.update_traces(
textposition = 'top right'
, line_color='#6495ED'
)

# fig.update_yaxes(rangemode="tozero")

fig.add_trace(go.Scatter(x=df_agg_week['date_vio_week'], y=df_agg_week['Defect rate'], line_color='#2E5894'))


fig.update_layout(
xaxis_title="Periods"
, yaxis_title="Defect Rate"
, showlegend=False
)

# fig.add_annotation(x='2024-06-16', y=5,
#             text="Text annotation with arrow",
#             showarrow=True,
#             arrowhead=1)

# fig.add_annotation(x='2024-06-16', y=4,
#             text="Text annotation without arrow",
#             showarrow=False,
#             yshift=-20)

### Defect rate for each feature period starting from bans to photocontrol to consider changes between dates

In [ ]:
df_defect_rate_quality = read_bq("""

""")


df_defect_rate_quality['defect_rate'] = (df_defect_rate_quality['violations'] / df_defect_rate_quality['rides_done']).round(3)

df_defect_rate_quality.head(10)

# Recording total datasets on Defect and Incident rates

In [ ]:
df_defect_rate_quality = df_defect_rate_quality.sort_values(['country_name_vio', 'date_vio_week'], ascending=True)
df_incident_rate_first = df_incident_rate_first.sort_values(['country_name', 'incident_date_week'], ascending=True)

In [ ]:
writing_excel('Incident&Defect_rates', df_incident_rate_diff, df_incident_rate_pivot, df_defect_rate_quality,)

# Incident rate report research (How to fix it seamlessly)

In [ ]:
df_incident_rate_first = read_bq("""

""")


df_incident_rate_first


In [ ]:
fig = px.line(df_incident_rate_first.groupby(['incident_date_week', 'country_name'], as_index=False)[['incident_rate_bi', 'incident_rate_initial']].agg('max').query(f"country_name in ('')")
            , x="incident_date_week"
            , y=['incident_rate_bi', 'incident_rate_initial']
            , title=f"Incident rate in {city}"
            , width=1500, height=600
            , markers=True
            )

fig.update_layout(
    xaxis_title="Dates"
            , yaxis_title="Incident rate"
)

fig.update_yaxes(rangemode="tozero")

fig.show()